In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## EDA

In [ ]:
game_data =  pd.read_csv("../input/nfl-big-data-bowl-2021/games.csv")
player_data = pd.read_csv("../input/nfl-big-data-bowl-2021/players.csv")
play_data = pd.read_csv("../input/nfl-big-data-bowl-2021/plays.csv")

# Combine weekly play data:

In [ ]:
week_data = pd.DataFrame()
for i in range(1,17):
    week = "week"+str(i)+".csv"
    df = pd.read_csv("../input/nfl-big-data-bowl-2021/"+week)
    week_data = pd.concat([df],ignore_index = True)

In [ ]:
week_data = week_data.filter(['nflId','gameId','playId'])
play_data = play_data.filter(['gameId', 'playId', 'passResult', 'penaltyCodes', 'playResult'])

In [ ]:
# test = player_data.merge(play_data, how = "inner", left_on = "nflId", right_on = "nflId")
player_data = week_data.merge(player_data, how = "inner", left_on = "nflId", right_on = "nflId")

In [ ]:
player_data

In [ ]:
offensive_positions = ["QB", "WR", "RB", "TE", "FB", "HB"]
defensive_positions = ["SS", "FS", "MLB", "CD", "LB", "OLB", "DE", "ILB", "S", "DB", "NT"]

In [ ]:
player_plays = pd.merge(player_data, play_data, how = 'left', left_on =['gameId','playId'], right_on = ['gameId', 'playId'])
player_plays

In [ ]:
# avg_player_play_result = player_plays.groupby(['displayName', 'position', 'height', 'weight', 'birthDate', 'collegeName'])['playResult'].mean().reset_index()
avg_player_play_result = player_plays.groupby(['displayName', 'position', 'height', 'weight', 'birthDate'])['playResult'].mean().reset_index()

In [ ]:
avg_player_play_result

In [ ]:

defensive_avg_play_result = avg_player_play_result.loc[avg_player_play_result['position'].isin(defensive_positions)]
offensive_avg_play_result = avg_player_play_result.loc[avg_player_play_result['position'].isin(offensive_positions)]
defensive_avg_play_result

In [ ]:
from dateutil.parser import parse
from datetime import datetime, date

defensive_avg_play_result = defensive_avg_play_result.rename(columns = {'birthDate':'Age'})

def formatBirthDate(row):
    dt = parse(str(row['Age']))
    return dt.strftime('%Y-%m-%d')

defensive_avg_play_result['Age'] = defensive_avg_play_result.apply(formatBirthDate, axis=1)

def calculateAge(birth):
    birth = datetime.strptime(birth, '%Y-%m-%d').date()
    today = date.today()
    return today.year - birth.year - ((today.month,today.day) < (birth.month,birth.day))

defensive_avg_play_result['Age'] = defensive_avg_play_result['Age'].apply(calculateAge)


def formatHeight(row):
    if row['height'][1] == "-":
        return int(row['height'][0]) * 12 + int(row['height'][2])
    else:
        return int(row['height'])
    
defensive_avg_play_result['height'] = defensive_avg_play_result.apply(formatHeight, axis=1)
    

In [ ]:
from sklearn.linear_model import LinearRegression

for position in defensive_positions:
    print(f'Defensive_avg_play_result_{position}')
    stats_groupedby_position = defensive_avg_play_result.loc[defensive_avg_play_result['position'] == position]
    cols_X = [col for col in stats_groupedby_position if col not in ['displayName', 'playResult', 'position']]
    cols_Y = [col for col in stats_groupedby_position if col in ['playResult']]
    stats_groupedby_position_X= stats_groupedby_position[cols_X]
    stats_groupedby_position_Y= stats_groupedby_position[cols_Y]
    if len(stats_groupedby_position_X) > 1:
        print("Sample size: " + str(len(stats_groupedby_position_X)))
        reg = LinearRegression().fit(stats_groupedby_position_X,stats_groupedby_position_Y)
        print("R^2 value: " + str(reg.score(stats_groupedby_position_X,stats_groupedby_position_Y)))
        print("Coefficients: " + str(reg.coef_))
    else:
        print("No data found for " + f'Defensive_avg_play_result_{position}')
    print("\n")